# Analisa Planos

### Importa os módulos necessários

In [134]:
import pandas as pd

### Importa o csv dos planos e mostra algumas informações

In [157]:
data = pd.read_csv('planos_pgls.csv')
data.head(), data.info(), data.nunique()
data['professores'] = data['professores'].apply(lambda x: x.split(', '))
data['email_professores'] = data['email_professores'].apply(lambda x: x.split(', '))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 695 entries, 0 to 694
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   id                 695 non-null    int64 
 1   ano                695 non-null    int64 
 2   periodo            695 non-null    int64 
 3   curso              695 non-null    object
 4   disciplina         695 non-null    object
 5   codigo_turma       695 non-null    object
 6   professores        695 non-null    object
 7   email_professores  695 non-null    object
 8   link               695 non-null    object
dtypes: int64(3), object(6)
memory usage: 49.0+ KB


### Organiza os planos com determinado parâmtro, podendo ser por curso ou disciplina

In [160]:
sort_param = input('Quais dados deseja filtrar? Escolha entre curso e disciplina').lower()
save = input('Deseja salvar em um csv? Y/N').lower()
sorted_data = pd.DataFrame()

if sort_param == 'curso':
    sorted_data = data[['id', 'ano', 'periodo', 'curso', 'disciplina', 'codigo_turma', 'link']].sort_values(by=['curso', 'periodo'])
elif sort_param == 'disciplina':
    sorted_data = data[['id', 'periodo', 'curso', 'disciplina', 'codigo_turma', 'link']].sort_values(by=['disciplina', 'periodo'])
else: 
    print('Erro: Digite Curso ou Disciplina para filtrar')

if save == 'y':
    sorted_data.to_csv(f'planos_pgls_sorted_{sort_param}.csv', index=False)

sorted_data

,id,periodo,curso,disciplina,codigo_turma,link
647,47559,31,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,ACTING,MBM04A,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
421,51941,32,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,ACTING,MBM05A,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
477,51803,32,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,ACTING,MBM05A_B,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
267,52871,33,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,ACTING,MBMTM06A,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
144,57543,34,MASTER IN BUSINESS MANAGEMENT - ESPECIALIZAÇÃO...,ACTING,MBMTM07A,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
...,...,...,...,...,...,...
542,50901,31,PROGRAMA AVANÇADO EM DATA SCIENCE E DECISÃO,WORKSHOP VII,PADSW08WKVII,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
372,52117,32,PROGRAMA AVANÇADO EM DATA SCIENCE E DECISÃO,WORKSHOP VII,PADSONL02WKVII,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
357,52325,32,PROGRAMA AVANÇADO EM DATA SCIENCE E DECISÃO,WORKSHOP VIII,PADSONL02WKVIII,https://cgi.insper.edu.br/DiarioDeClasse/Plano...
533,51247,31,PROGRAMA AVANÇADO EM GESTÃO PÚBLICA,ÉTICA E ORGANIZAÇÃO PÚBLICA,PAGPW12EOP,https://cgi.insper.edu.br/DiarioDeClasse/Plano...


In [164]:
sorted_data.groupby('disciplina')

### Faz o web scraping dos planos

In [30]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [198]:
for link in sorted_data['link']:
    # Faz uma requisição para obter a página HTML
    page = requests.get(link)
    # Faz um parse do conteúdo da página
    soup = BeautifulSoup(page.content, "html.parser")
    # Procura por todos os elementos da página que são uma div que possuem a classe textInline
    rows = soup.find_all('tr')
    print(link)
    if len(rows) > 0:
        for i in range(2, len(rows)-2):
            aula = rows[i].text.split('\n')
            print(aula)
        break

https://cgi.insper.edu.br/DiarioDeClasse/PlanoDeAula/PlanoDeAulaPDF.aspx?t=Plano%20de%20Aula%20Atualizado&id=47559,&d=false
['', 'Aula 1\r', '                                                ', '31/01/2024', '19:15-22:15', '', '', '', '', '', '', '', 'Relacionar as decisões organizacionais tomadas em relação a marcas, linhas de produtos e critérios de desempenho de operações com a estratégia da empresa. Analisar mudanças de mercado e prever seus impactos na gestão de demanda e de capacidade de operações.', 'Introdução ao módulo Acting - Marketing e Operações', 'APS Visão Sistêmica da Organização', 'Discussão plenária sobre o temaAtividade individual reflexiva sobre visão sistêmica', '']
['', 'Aula 2\r', '                                                ', '01/02/2024', '19:15-22:15', '', '', '', '', '', '', '', 'Relacionar as decisões organizacionais tomadas em relação a marcas, linhas de produtos e critérios de desempenho de operações com a estratégia da empresa.', 'Fundamentos de Opera